In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
feature_data_dicts = PROJECT_JSON["feature_data"]

for data_name, data_dict in feature_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
sample_data_dicts = PROJECT_JSON["sample_data"]

for data_name, data_dict in sample_data_dicts.items():

    print(data_name)

    data_dict["dataframe"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = PROJECT_JSON["sample_alias"]

In [ ]:
for w_or_h, data_dicts, factor_x_element in (
    ("w", feature_data_dicts, w.T),
    ("h", sample_data_dicts, h),
):

    for factor_name, data_name_indices in PROJECT_JSON[
        "{}_summary".format(w_or_h)
    ].items():

        data_dicts_ = {}

        score_moe_p_value_fdr_dicts = {}

        for data_name, indices in data_name_indices.items():

            data_dicts_[data_name] = {
                "dataframe": data_dicts[data_name]["dataframe"].loc[indices]
            }

            score_moe_p_value_fdr_dicts[data_name] = pd.read_csv(
                os.path.join(
                    PATH["h|match/"],
                    factor_name,
                    "{}.tsv".format(kraft.normalize_file_name(data_name)),
                ),
                sep="\t",
                index_col=0,
            )

        kraft.make_match_panel_summary(
            factor_x_element.loc[factor_name],
            data_dicts_,
            score_moe_p_value_fdr_dicts,
            plot_only_shared_by_target_and_all_data=PROJECT_JSON[
                "plot_only_shared_by_target_and_all_data"
            ],
            plot_std=PROJECT_JSON["plot_std"],
            layout={
                "title": {
                    "text": PROJECT_JSON["factor_name"].get(factor_name, factor_name)
                }
            },
            html_file_path=os.path.join(
                PATH["h|summary/"], "{}.html".format(factor_name)
            ),
        )